Notebook simulating our active learning loop

In [ ]:
#colab
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/aml_final/aml_final/
! git pull
! pip install setfit
'''
from datasets import load_dataset, concatenate_datasets
from setfit import TrainingArguments

from data.dataset_config import DatasetConfig
from train.active_learning import ActiveTrainer, create_random_subset, label_sentences
from train.active_learning_config import ActiveLearningConfig
from train.metrics import comprehensive_metrics


In [ ]:
dataset = load_dataset("dair-ai/emotion")
train_dataset = dataset["train"]
dataset_config = DatasetConfig(text_column="text", num_classes=6)
active_config = ActiveLearningConfig(active_learning_cycles=1, samples_per_cycle=10, model_name="sentence-transformers/all-mpnet-base-v2")
intermediate_args = TrainingArguments(num_iterations=10, num_epochs=(1,8))
final_args = TrainingArguments(num_iterations=20, num_epochs=(1,16))

inittial_subset = create_random_subset(train_dataset, dataset_config, num_samples=50)
after_train_callback = lambda trainer, **kwars: print(trainer.evaluate())
active_trainer = ActiveTrainer(
    full_train_dataset=train_dataset, 
    train_args=intermediate_args,
    final_model_train_args=final_args,
    active_learning_config=active_config,
    dataset_config=dataset_config,
    eval_dataset=dataset["validation"], 
    metric=comprehensive_metrics, 
    initial_train_subset=inittial_subset
    )

In [ ]:
intermediate_model = active_trainer.run_training().model
selected_sentences = active_trainer.select_sentences(intermediate_model)
print(selected_sentences)

In [ ]:
labeled_sentences = label_sentences(selected_sentences, labeled_dataset=train_dataset, text_column="text")
print(labeled_sentences["text"])
print(label_sentences["label"])

In [ ]:
new_dataset = concatenate_datasets([active_trainer.train_subset, labeled_sentences])
active_trainer.train_subset = new_dataset

final_model = active_trainer.run_training(final_model=True).model

In [ ]:
final_model.predict(["Great presentation!", "Very suprising results!"])